In [1]:
pip install openai==0.28

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.8 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.54.5
    Uninstalling openai-1.54.5:
      Successfully uninstalled openai-1.54.5


In [2]:
import torch
import torchaudio
from torchaudio.utils import download_asset
import IPython
import numpy as np
import os
from transformers import pipeline
import csv
import openai
import pandas as pd
import re
import csv
import json

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
pipe = pipeline(
  "automatic-speech-recognition",
  model="openai/whisper-tiny",
  chunk_length_s=15,
  device=device,
)

def load_custom_audio(file_path):
    waveform, sample_rate = torchaudio.load(file_path)
    return {"array": waveform[0].numpy(), "sampling_rate": sample_rate}

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.98k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/151M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.75k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

...

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

In [4]:
audio_file_path = "/content/naman.m4a"
sample = load_custom_audio(audio_file_path)

prediction = pipe(sample.copy(), batch_size=8)["text"]
print("Transcription:", prediction)

/usr/local/lib/python3.10/dist-packages/transformers/models/whisper/generation_whisper.py:509: FutureWarning: The input name `inputs` is deprecated. Please make sure to use `input_features` instead.
  warnings.warn(
Due to a bug fix in https://github.com/huggingface/transformers/pull/28687 transcription using a multilingual Whisper will default to language detection followed by transcription instead of translation to English.This might be a breaking change for your use case. If you want to instead always translate your audio to English, make sure to pass `language='en'`.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.43.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `

Transcription:  Yes, starting with my intro, my name is Namangupta. I am pressing bit of bit of from J.K. L.F. to University. With our tremendous CGPA of 8.6 talking about my experience. I have done my internship from LM&IT where we used to learn project about machine landing. The machine landing project which I created in my internship was movie recommendation system in which I used MF-based techniques and created our recommendation system. Secondly, I have created a project Fishing Your Alp Editor which use fully connected network in which I have applied my deep-dunning concepts and fishing your predictor use a re-lo-function for its identliates and finally, our will have sigmoid function within accuracy of 85.2. It is a very important feature. in my gate 2024 data science and artificial intelligence. Secondly, I have gold medal in python at hacker rank. Third, I have solved more than 200 plus gsa In python add haker rank, third I have solved more than 200 plus dsa coding quotients a

In [18]:
openai.api_key = ********************************************************************************
def extract_details_from_text(text):
    # prompt = f"Extract the following details from the text: name, age, skills, goals, and provide a rating out of five based on the person's overall knowledge and skills: \n\n {text}."
    prompt = f"""
    I have an interview transcript:\n{text}
    and I need to extract key candidate details that are important for an interviewer. Please analyze the input and provide the following details in a structured JSON format and if not mentioned keep it empty:
    Candidate's Full Name
    Position/Role applied for
    Contact Information (Phone, Email)
    Educational Background (Degree and University, Year, CGPA)
    Work Experience (Previous Companies, Roles, Duration)
    Skills and Technologies (Programming Languages, Tools, Certifications)
    Strengths or Key Attributes (as mentioned by the candidate)
    Reason for Applying (if mentioned)
    Candidate's Availability (Immediate, Notice Period)
    Any Other Relevant Information (Awards, Achievements, Languages Spoken)
    """
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=400,
        temperature=0.5)
    return response['choices'][0]['message']['content']


# Extract details using OpenAI
extracted_text = extract_details_from_text(prediction)
print(extracted_text)
candidate_data = json.loads(extracted_text)
# print(json.dumps(candidate_data, indent=4))  # Pretty print the candidate data




{
  "Candidate's Full Name": "Namangupta",
  "Position/Role applied for": "",
  "Contact Information": {
    "Phone": "9166712583",
    "Email": "n65712583"
  },
  "Educational Background": {
    "Degree": "J.K. L.F. to University",
    "Year": "",
    "CGPA": "8.6"
  },
  "Work Experience": {
    "Previous Companies": "LM&IT",
    "Roles": "Intern",
    "Duration": ""
  },
  "Skills and Technologies": {
    "Programming Languages": "Python",
    "Tools": "Machine Learning, Deep Learning",
    "Certifications": "Gold Medal in Python at HackerRank"
  },
  "Strengths or Key Attributes": "Created a movie recommendation system using MF-based techniques, Deep learning concepts, and fully connected network",
  "Reason for Applying": "",
  "Candidate's Availability": "",
  "Any Other Relevant Information": {
    "Awards": "Gold Medal in Python at HackerRank",
    "Achievements": "Gate 2024 Data Science and Artificial Intelligence, Solved 200+ DSA coding questions on HackerRank",
    "Language

In [19]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_relevance(candidate_data, job_requirements):
  # Convert text to TF-IDF vectors
  vectorizer = TfidfVectorizer()
  vectors = vectorizer.fit_transform([candidate_data, job_requirements])

  # Calculate cosine similarity
  cos_sim = cosine_similarity(vectors[0], vectors[1])
  return cos_sim[0][0]

# Example job requirements (this would typically come from a job description)
job_requirements = '''{
    "Skills": ["Python"],
    "Roles": ["Intern"],
    "Certifications": ["AWS Certified", "Google Cloud Certified"]}
    '''

relevance_score = calculate_relevance(extracted_text, job_requirements)
print(f"Relevance Score: {relevance_score}")

Relevance Score: 0.09857890016413245


In [20]:
# Flatten the nested dictionaries into a single-level dictionary
flattened_data = {
    "Candidate's Full Name": candidate_data["Candidate's Full Name"],
    "Position/Role applied for": candidate_data["Position/Role applied for"],
    "Degree": candidate_data["Educational Background"]["Degree"],
    # "University": candidate_data["Educational Background"]["University"],
    "Year": candidate_data["Educational Background"]["Year"],
    "Previous Companies": candidate_data["Work Experience"]["Previous Companies"],
    "Roles": candidate_data["Work Experience"]["Roles"],
    "Duration": candidate_data["Work Experience"]["Duration"],
    "Programming Languages": candidate_data["Skills and Technologies"]["Programming Languages"],
    "Tools": candidate_data["Skills and Technologies"]["Tools"],
    "Strengths or Key Attributes": candidate_data["Strengths or Key Attributes"],
    "Reason for Applying": candidate_data["Reason for Applying"],
    "Candidate's Availability": candidate_data["Candidate's Availability"],
    "Any Other Relevant Information": candidate_data["Any Other Relevant Information"],
    "Relevance Score": relevance_score
}
# Define the CSV file path
csv_file = 'candidate_data.csv'

# Check if the file exists
file_exists = os.path.isfile(csv_file)

# Open the CSV file in append mode, or create it if it doesn't exist
with open(csv_file, mode='a', newline='') as file:
    fieldnames = flattened_data.keys()
    writer = csv.DictWriter(file, fieldnames=fieldnames)

    # If the file doesn't exist, write the header
    if not file_exists:
        writer.writeheader()

    # Write the candidate data to the CSV
    writer.writerow(flattened_data)

print(f"Data saved to {csv_file}")

Data saved to candidate_data.csv
